In [1]:
def composite_ipr(p_wf, p_r, p_bubble, PI, w_c):
    """
    Calculate the oil and total liquid flow rates of an oil well using the Composite Inflow Performance Relationship (IPR).
    This IPR extends Vogel's equation to account for gas generated below the bubble point and includes water cut correction.
    When the bottomhole pressure is below the bubble point, the function interpolates between the Vogel IPR (zero water cut)
    and the PI curve (100% water cut) based on the water cut.

    Parameters:
    p_wf : float
        Bottomhole flowing pressure (psia)
    p_r : float
        Average reservoir pressure (psia)
    p_bubble : float
        Bubble point pressure (psia)
    PI : float
        Productivity Index (STB/day/psi)
    w_c : float
        Water cut (fraction between 0 and 1)

    Returns:
    q_o : float
        Oil flow rate at bottomhole flowing pressure p_wf (STB/day)
    q_L : float
        Total liquid flow rate at bottomhole flowing pressure p_wf (STB/day)
    """
    # Validate input pressures
    if p_wf < 0 or p_r <= 0 or p_bubble <= 0:
        raise ValueError("Pressures must be positive values.")
    if p_wf > p_r:
        raise ValueError("Bottomhole flowing pressure must be less than or equal to reservoir pressure.")
    if not 0 <= w_c <= 1:
        raise ValueError("Water cut must be between 0 and 1.")

    # Above bubble point pressure
    if p_wf >= p_bubble:
        # Use PI method
        q_L = PI * (p_r - p_wf)
        q_o = q_L * (1 - w_c)
    else:
        # Below bubble point pressure
        # Calculate flow rate at bubble point pressure using PI method
        q_bubble = PI * (p_r - p_bubble)

        # Calculate oil flow rate using Vogel's equation
        p_ratio = p_wf / p_bubble
        q_vogel_oil = q_bubble * (1 - 0.2 * p_ratio - 0.8 * p_ratio**2)

        # Calculate total liquid flow rate using PI method
        q_PI_liquid = PI * (p_r - p_wf)

        # Interpolate between Vogel's IPR and PI curve based on water cut
        q_L = (1 - w_c) * q_vogel_oil + w_c * q_PI_liquid

        # Calculate oil flow rate
        q_o = q_L * (1 - w_c)

    return q_o, q_L

def calculate_PI(q1, p_r, p_wf1):
    """
    Calculate the Productivity Index (PI) using known flow rate q1 at p_wf1.

    Parameters:
    q1 : float
        Known flow rate at bottomhole flowing pressure p_wf1 (STB/day)
    p_r : float
        Average reservoir pressure (psia)
    p_wf1 : float
        Known bottomhole flowing pressure (psia)

    Returns:
    PI : float
        Productivity Index (STB/day/psi)
    """
    # Validate input pressures
    if p_wf1 < 0 or p_r <= 0:
        raise ValueError("Pressures must be positive values.")
    if p_wf1 > p_r:
        raise ValueError("Bottomhole flowing pressure must be less than or equal to reservoir pressure.")

    # Calculate PI using known data
    delta_p = p_r - p_wf1
    if delta_p == 0:
        raise ValueError("Pressure difference cannot be zero.")
    PI = q1 / delta_p
    return PI


In [ ]:

# Example usage:
if __name__ == "__main__":
    # Given parameters
    p_r = 2550       # Reservoir pressure in psia
    p_bubble = 2100  # Bubble point pressure in psia
    p_wf = 2300      # Bottomhole flowing pressure in psia
    q1 = 500         # Known flow rate at p_wf1
    p_wf1 = 2300     # Known bottomhole flowing pressure in psia
    w_c = 0.5        # Water cut (20%)

    # Calculate PI using known flow rate
    PI = calculate_PI(q1, p_r, p_wf1)
    print(f"Calculated PI: {PI:.4f} STB/day/psi")

    # Calculate flow rates at desired bottomhole pressure
    q_o, q_L = composite_ipr(p_wf, p_r, p_bubble, PI, w_c)
    print(f"Oil flow rate at p_wf = {p_wf} psia: {q_o:.2f} STB/day")
    print(f"Total liquid flow rate at p_wf = {p_wf} psia: {q_L:.2f} STB/day")
